In [16]:
import numpy as np
from scipy.io import loadmat

# 读取mat文件
mat_data = loadmat(r'D:\神经生物学\20250513核团放电\20250513核团放电v2.mat')

# 去除mat文件自带的无关字段
mat_data = {k: v for k, v in mat_data.items() if not k.startswith('__')}

# print(mat_data.keys())

In [17]:
mat_data['comtext'][4] = 5000
mat_data['data'] = mat_data['data']*1000 

# 提取com的第三列
com_third_col = mat_data['com'][:, 2].astype(int)

# 用comtext中的顺序替换第五列
comtext = np.array(mat_data['comtext']).flatten().astype(str)
new_fifth_col = []
for idx in mat_data['com'][:, 4].astype(int):
    # 检查idx是否为整数且在comtext范围内
    if isinstance(idx, (int, np.integer)) and 0 < idx <= len(comtext):
        new_fifth_col.append(comtext[int(idx)-1])
    else:
        new_fifth_col.append(idx)
new_fifth_col = np.array(new_fifth_col)

# 组合成anotation
anotation = np.column_stack((new_fifth_col, com_third_col))
# print(anotation)

# 加入到mat_data
mat_data['anotation'] = anotation

In [18]:
# 提取需要的字段
keys_to_keep = ['data', 'anotation']
mat_data_filtered = {key: mat_data[key] for key in keys_to_keep if key in mat_data}

# 按照anotation将数据分开
for i in range(len(mat_data_filtered['anotation'][:, 0])):
    if mat_data_filtered['anotation'][i, 0] == 'down':
        mat_data_filtered['anotation'][i, 0] = f"down_{mat_data_filtered['anotation'][i - 1, 0]}-{mat_data_filtered['anotation'][i + 1, 0]}"

# 展示mat_data_filtered的内容
print("mat_data_filtered的内容：")
for k, v in mat_data_filtered.items():
    print(f"key: {k}")
    print(f"  type: {type(v)}")
    if isinstance(v, np.ndarray):
        print(f"  shape: {v.shape}")
        print(f"  data (前10项): {v.flatten()[:10]}")
    else:
        print(f"  value: {v}")
    print("-" * 30)

# 保存为npz文件
save_dir = r'D:\神经生物学\20250513核团放电\nuc_v2.npz'
np.savez(save_dir, **mat_data_filtered)
print(f"数据已保存到{save_dir}")


mat_data_filtered的内容：
key: data
  type: <class 'numpy.ndarray'>
  shape: (1, 3357850)
  data (前10项): [-0.7997664 -0.7997664 -0.7997664 -0.7997664 -0.7997664 -0.7997664
 -0.7997664 -0.7997664 -0.7997664 -0.7997664]
------------------------------
key: anotation
  type: <class 'numpy.ndarray'>
  shape: (41, 2)
  data (前10项): ['2000' '15349' 'down_2000-3000' '57449' '3000' '86849' 'down_3000-4000'
 '136899' '4000' '194799']
------------------------------
数据已保存到D:\神经生物学\20250513核团放电\nuc_v2.npz


In [19]:
# 按照anotation将数据分开
mat_data_split = {}
for i in range(len(mat_data_filtered['anotation'][:, 0])):
    if i < len(mat_data_filtered['anotation']) - 1:
        mat_data_split[mat_data_filtered['anotation'][i, 0]] = mat_data_filtered['data'].flatten()[mat_data_filtered['anotation'][i, 1].astype(int) :mat_data_filtered['anotation'][i + 1, 1].astype(int)]
    else:
        mat_data_split[mat_data_filtered['anotation'][i, 0]] = mat_data_filtered['data'].flatten()[mat_data_filtered['anotation'][i, 1].astype(int):]

# 展示mat_data_split的内容
print("mat_data_split的内容：")
for k, v in mat_data_split.items():
    print(f"key: {k}")
    print(f"  type: {type(v)}")
    if isinstance(v, np.ndarray):
        print(f"  shape: {v.shape}")
        print(f"  data (前10项): {v.flatten()[:10]}")
    else:
        print(f"  value: {v}")
    print("-" * 30)



# 保存为npz文件
save_split_dir = r'D:\神经生物学\20250513核团放电\nuc_v2_split.npz'
np.savez(save_split_dir, **mat_data_split)
print(f"数据已保存到{save_split_dir}")

mat_data_split的内容：
key: 2000
  type: <class 'numpy.ndarray'>
  shape: (42100,)
  data (前10项): [0.10475174 0.10249726 0.09854667 0.09328169 0.08712772 0.0806115
 0.0742902  0.06862214 0.06393846 0.06042403]
------------------------------
key: down_2000-3000
  type: <class 'numpy.ndarray'>
  shape: (29400,)
  data (前10项): [0.08702061 0.09440139 0.10263713 0.11133559 0.1200583  0.12824787
 0.13551937 0.14151536 0.14594102 0.14860952]
------------------------------
key: 3000
  type: <class 'numpy.ndarray'>
  shape: (50050,)
  data (前10项): [0.11441495 0.1137313  0.11315896 0.11288304 0.11294043 0.11336645
 0.11413968 0.11509633 0.11593288 0.11623146]
------------------------------
key: down_3000-4000
  type: <class 'numpy.ndarray'>
  shape: (57900,)
  data (前10项): [ 0.00615777  0.00503874  0.00347796  0.0013716  -0.00135985 -0.00466465
 -0.00847507 -0.01267049 -0.01709886 -0.02160573]
------------------------------
key: 4000
  type: <class 'numpy.ndarray'>
  shape: (52850,)
  data (前10项): [

In [20]:
# 检查数据
print("检查数据：")
split_data = np.load(save_split_dir, allow_pickle=True)
print(split_data.files)


检查数据：
['2000', 'down_2000-3000', '3000', 'down_3000-4000', '4000', 'down_4000-5000', '5000', 'down_5000-5500', '5500', 'down_5500-6000', '6000', '6500', 'down_6500-6800', '6800', '6900', '6950', '7000', 'down_7000-7060', '7060', '7120', '7200', 'down_7200-7290', '7290', '7357', 'down_7357-7431', '7431', 'down_7431-7500', '7500', 'down_7500-7600', '7600', 'down_7600-7700', '7700', 'down_7700-7800', '7800', 'down_7800-7900', '7900', 'down_7900-down', 'down_down_7900-down-8', '8500', 'down_8500-9000', '9000']
